In [14]:
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
sns.set_style("darkgrid")
warnings.filterwarnings('ignore')

In [15]:
folder = ''
temp1 = pd.read_csv(folder+'Task1_step3.csv')
temp2 = pd.read_csv(folder+'Task2.csv')
temp3 = pd.read_csv(folder+'Task3.csv')
print(temp1.columns)
temp1.head()

Index(['participant_ID', 'treatment', 'part', 'trial_no', 'rounds',
       'red_prior_prob', 'true_guilty_suspect', 'suspect_accused', 'choice_1',
       'choice_2', 'choice_3', 'choice_4', 'choice_5', 'choice_6', 'choice_7',
       'choice_8', 'choice_9', 'outcome_1', 'outcome_2', 'outcome_3',
       'outcome_4', 'outcome_5', 'outcome_6', 'outcome_7', 'outcome_8',
       'outcome_9', 'timing_choice_1', 'timing_choice_2', 'timing_choice_3',
       'timing_choice_4', 'timing_choice_5', 'timing_choice_6',
       'timing_choice_7', 'timing_choice_8', 'timing_choice_9',
       'timing_choice_10', 'timing_choice_11', 'correct_suspect_accused',
       'evidence_round'],
      dtype='object')


participant_ID  treatment  part  trial_no  rounds  red_prior_prob  \
0               1          0     1         1     3.0             0.9   
1               1          0     1         2     3.0             0.1   
2               1          0     1         3     1.0             0.1   
3               1          0     1         4     7.0             0.1   
4               1          0     1         5     5.0             0.1   

   true_guilty_suspect  suspect_accused  choice_1  choice_2  ...  \
0                    0                0       0.0       0.0  ...   
1                    1                1       0.0       0.0  ...   
2                    1                1       0.0       NaN  ...   
3                    1                1       0.0       0.0  ...   
4                    1                1       0.0       0.0  ...   

   timing_choice_4  timing_choice_5  timing_choice_6  timing_choice_7  \
0       13003.3880       14173.5518              NaN              NaN   
1        6009.1745        7012.6483              NaN              NaN   
2              NaN              NaN              NaN              NaN   
3        3992.2255        4492.2955        5009.0345        5525.7735   
4        3508.8245        4042.2325        4658.9855        5662.4593   

   timing_choice_8  timing_choice_9  timing_choice_10  timing_choice_11  \
0              NaN              NaN               NaN               NaN   
1              NaN              NaN               NaN               NaN   
2              NaN              NaN               NaN               NaN   
3        7442.8681        8446.4749               NaN               NaN   
4              NaN              NaN               NaN               NaN   

   correct_suspect_accused  evidence_round  
0                        1               0  
1                        1               0  
2                        1               0  
3                        1               0  
4                        1               0  

[5 rows x 39 columns]

In [16]:
data = []
for key,value in temp1.iterrows():
    info_b = value[:6].to_list()
    info_e = value[6:8].to_list()
    rounds = value[8:17].to_list()
    rounds = [x for x in rounds if x == 1.0 or x == 0.0]
    suspect = []
    post = value[5] if value[1] == 0 else 1-value[5]
    pg = 0.75 if value[1] == 0 else 1
    pi = 1 if value[1] == 0 else 0.75
    e_f = 0
    e_s = -1
    for i in range(len(rounds)):
        suspect.append(rounds[i])
        e_f = int(value[17+i]) if e_f != 1 else 1
        if e_f == 1: 
            e_s = value[7]
        else: e_f = 1 if e_f == 1 and i != 0 else 0
        temp = [i+1, 0, suspect[-1], suspect[:-1].count(0.0), 
               suspect[:-1].count(1.0), e_f, e_s]
        data.append(info_b+temp+[post]+info_e+[temp1['timing_choice_'+str(i+1)][key]])
        post = post*pg/(post*pg + pi*(1-post))
        post = value[7] if e_f == 1 else post
    suspect.append(value[7])
    temp = [i+1, 1, suspect[-1], suspect[:-1].count(0.0), suspect[:-1].count(1.0), e_f, e_s]
    data.append(info_b+temp+[post]+info_e+[temp1['timing_choice_'+str(i+1)][key]])

In [17]:
cols = ['participant_ID', 'treatment', 'part', 'trial_no', 'rounds',
       'red_prior_prob', 'current_rounds', 'action_type', 'suspect', 
        'count_red_samples', 'count_blue_samples', 'evidence_found',
       'evidence_suspect', 'posterior', 'true_guilty_suspect', 'suspect_accused',
       'time']
df2 = pd.DataFrame(data,columns=cols)
df2.head()

participant_ID  treatment  part  trial_no  rounds  red_prior_prob  \
0             1.0        0.0   1.0       1.0     3.0             0.9   
1             1.0        0.0   1.0       1.0     3.0             0.9   
2             1.0        0.0   1.0       1.0     3.0             0.9   
3             1.0        0.0   1.0       1.0     3.0             0.9   
4             1.0        0.0   1.0       2.0     3.0             0.1   

   current_rounds  action_type  suspect  count_red_samples  \
0               1            0      0.0                  0   
1               2            0      0.0                  1   
2               3            0      0.0                  2   
3               3            1      0.0                  3   
4               1            0      0.0                  0   

   count_blue_samples  evidence_found  evidence_suspect  posterior  \
0                   0               0              -1.0   0.900000   
1                   0               0              -1.0   0.870968   
2                   0               0              -1.0   0.835052   
3                   0               0              -1.0   0.791531   
4                   0               0              -1.0   0.100000   

   true_guilty_suspect  suspect_accused       time  
0                  0.0              0.0  6902.4234  
1                  0.0              0.0  7402.4234  
2                  0.0              0.0  7902.4248  
3                  0.0              0.0  7902.4248  
4                  1.0              1.0  3208.7825

In [18]:
df2.to_csv('Task1_step4.csv', index=False)